In [ ]:
using Distributed
addprocs(5);

In [ ]:
using LinearAlgebra
using SparseArrays
using Plots

In [ ]:
using Revise
using LatticeQM

In [ ]:
using ProgressMeter; ProgressMeter.ijulia_behavior(:clear);

## Calculation

In [ ]:
lat = Geometries2D.honeycomb()
sx, sy, sz, sublA, sublB = getoperator(lat, ["SX", "SY", "SZ", "sublatticeAspin", "sublatticeBspin"])

hops = Operators.graphene(lat; mode=:spinhalf)
# Operators.addzeeman!(hops, lat, r->sign(r[4]-0.5).*0.5.*[sin(0.0π),0,cos(0.0π)] )

ks = kpath(lat; num_points=200)

nothing

In [ ]:
ks = kpath(lat; num_points=200)
bands = getbands(hops, ks, sz)
p1 = plot(bands; markersize=2, size=(300,200))

In [ ]:
hopsBDG = BdGOperator(hops)
Operators.setfilling!(hopsBDG, 0.48)

eOP = Superconductivity.electron(hopsBDG)

ks = kpath(lat; num_points=200)
bands = getbands(hopsBDG.h, ks, eOP.h)
p1 = plot(bands; markersize=2, size=(300,200))

In [ ]:
hopsBDG = BdGOperator(hops)
# v = gethubbard(lat; mode=:σx, a=0.5, U=-2.0) # interaction potential
v = Meanfield.getshortrangedpotential(lat, -2.4, 0.1)

ρ0_init = initialguess(v, :random; kind=:nonlocal) # initial guess
Δ0_init = initialguess(v, :random; kind=:nonlocal) # initial guess
ρ_init = BdGOperator(ρ0_init, Δ0_init)

ρ_sol, ϵ_GS, HMF, converged, error = Meanfield.solvehartreefock( # run the calculation
    hopsBDG, v, ρ_init, 0.48; klin=10, iterations=800, tol=1e-5,# p_norm=Inf,
    T=0.01, β=0.3,  show_trace=true, clear_trace=true
)
Operators.addchemicalpotential!(HMF.h, -HMF.μ)

nothing

In [ ]:
hmf = HMF.h
eOP = Superconductivity.electron(hmf)

bands_mf = getbands(hmf.h, ks, eOP.h)
# bands_mf.bands .-= HMF.μ # shift chemical potential to zero
p2 = plot(bands_mf; markersize=2, size=(300,200))

In [ ]:
BANDGAP = Spectrum.bandgap(DenseHops(hmf.h); klin=30)

In [ ]:
M, SC = Operators.localobservables(ρ_sol, lat)
M = round.(real(M); digits=8)
SC = round.(SC; digits=8)

@info "Local density" M[1,:]
@info "Local magnetization" M[2,:] M[3,:] M[4,:]
@info "Local pair correlations (s-wave)" SC[1,:]
@info "Local pair correlations (non-swave)" SC[2,:] SC[3,:] SC[4,:]

## Scanning parameters

In [ ]:
# using SharedArrays

lat = Geometries2D.honeycomb()
ks = kpath(lat; num_points=200)

hops = Operators.graphene(lat; mode=:spinhalf)
hopsBDG = BdGOperator(hops)

Us = -LinRange(0,3.5,10)
Vs = LinRange(0,1.0,10)
filling = 0.48
klin = 10

BANDGAP = zeros(length(Us),length(Vs))
SWAVEAUP = zeros(length(Us),length(Vs))
SWAVEBUP = zeros(length(Us),length(Vs))
SWAVEADOWN = zeros(length(Us),length(Vs))
SWAVEBDOWN = zeros(length(Us),length(Vs))
WAVEdiff = zeros(length(Us),length(Vs))
DENSITY0 = zeros(length(Us),length(Vs))
DENSITYdiff = zeros(length(Us),length(Vs))
# BANDGAP = SharedArray(BANDGAP)
# SWAVE0 = SharedArray(SWAVE0)
# SWAVEdiff = SharedArray(SWAVEdiff)
# DENSITY0 = SharedArray(DENSITY0)
# DENSITYdiff = SharedArray(DENSITYdiff)

@showprogress for i_=1:length(Us), j_=1:length(Vs)
 
    U = Us[i_]; V = Vs[j_]
    v = Meanfield.getshortrangedpotential(lat, U, V)

    ρ0_init = initialguess(v, :random; kind=:nonlocal)
    Δ0_init = initialguess(v, :random; kind=:nonlocal)
    ρ_init = BdGOperator(ρ0_init, Δ0_init)

    ρ_sol, ϵ_GS, HMF, converged, error = Meanfield.solvehartreefock( # run the calculation
        hopsBDG, v, ρ_init, filling; klin=klin, iterations=1000, tol=1e-5,# p_norm=Inf,
        T=0.01, β=0.35,  show_trace=false, clear_trace=false
    )
    Operators.addchemicalpotential!(HMF.h, -HMF.μ)
    
    M, SC = Operators.localobservables(ρ_sol, lat)
    M = round.(real(M); digits=10)
    SC = round.(SC; digits=10)
    
    # save the data of interest
    BANDGAP[i_,j_] = Spectrum.bandgap(DenseHops(HMF.h.h); klin=30)
    SWAVEAUP[i_,j_] = abs(SC[1,1]+SC[4,1])
    SWAVEBUP[i_,j_] = abs(SC[1,2]+SC[4,2])
    SWAVEADOWN[i_,j_] = abs(SC[1,1]-SC[4,1])
    SWAVEBDOWN[i_,j_] = abs(SC[1,2]-SC[4,2])
    DENSITY0[i_,j_] = (M[1,1]+M[1,2])/2
    DENSITYdiff[i_,j_] = (M[1,1]-M[1,2])/2
end

In [ ]:
p1 = heatmap(-Us,Vs, log.(BANDGAP'); clims=(-5,0), xlabel="-U", ylabel="V", title="Bandgap", size=(300,300))
p2 = heatmap(-Us,Vs,log.(abs.(DENSITYdiff')); clims=(-5,0), xlabel="-U", ylabel="V", title="Density A-B", size=(300,300))
p3 = heatmap(-Us,Vs, log.(SWAVEAUP+SWAVEADOWN+SWAVEBUP+SWAVEBDOWN)'; clims=(-4,0), xlabel="-U", ylabel="V", title="S-WAVE (A+B)-(UP+DOWN)", size=(300,300))
p4 = heatmap(-Us,Vs, log.(abs.(SWAVEAUP+SWAVEADOWN-(SWAVEBUP+SWAVEBDOWN)))'; clims=(-4,0), xlabel="-U", ylabel="V", title="S-WAVE |A-B|-(UP+DOWN)", size=(300,300))

p5 = heatmap(-Us,Vs, log.(SWAVEAUP+SWAVEBUP)'; clims=(-5,0),  xlabel="-U", ylabel="V", title="S-WAVE (A+B)-UP", size=(300,300))
p6 = heatmap(-Us,Vs,(SWAVEBUP-SWAVEBUP)'; xlabel="-U", ylabel="V", title="S-WAVE (A-B)-UP", size=(300,300))
p7 = heatmap(-Us,Vs, log.(SWAVEADOWN+SWAVEBDOWN)'; clims=(-5,0), xlabel="-U", ylabel="V", title="S-WAVE (A+B)-DOWN", size=(300,300))
p8 = heatmap(-Us,Vs,(SWAVEBDOWN-SWAVEBDOWN)'; xlabel="-U", ylabel="V", title="S-WAVE (A-B)-DOWN", size=(300,300))

p = plot(p1,p2,p3,p4,p5,p6,p7,p8; layout=(4,2), size=(600,900))
display(p)

# p1 = heatmap(-Us,Vs,DENSITY0; xlabel="-U", ylabel="V", title="Density A+B", size=(300,300))
# p = plot(p1,p2; layout=(1,2), size=(500,200))
# display(p)

nothing